# Dial-A-Ride Problem (DARP)

## Problem definition

In this exercice, we imagine you are running a Dial-A-Ride service. Your clients send you a request which specifies a pick-up and drop-off location, as well as a deadline before which they wish to be at their destination. For instance, you could have the situation shown below. The blue stick figures represent the pick-up locations, the red circles represent the drop-off locations, and the green house is the starting position of your vehicle (the _depot_).

![Example of DARP with 4 requests](DARPproblem.svg)

Your vehicle has two passenger seats. Your job is to plan the route you are going to take, in such a way that every request is fulfilled before its deadline. In other words, you must decide in which order to visit the pick-up and drop-off locations, such that every customer is picked up and dropped off in time, and the maximal capacity of the vehicle is never exceeded. Here the green arrows show a possible solution for our small example. Each green arrow also shows the travel time in minutes, you can check that all the deadlines are respected. 

![Possible solution for the DARP example](DARPsolution.svg)


## miniCP Model for DARP

You will now build a miniCP model to solve any DARP instance, step by step. First, we load the miniCP library and import the necessary classes. These will be useful later.

In [3]:
%jars ./

import minicp.engine.constraints.Circuit;
import minicp.engine.constraints.Element1DVar;
import minicp.engine.core.IntVar;
import minicp.engine.core.Solver;
import minicp.search.DFSearch;
import minicp.search.SearchStatistics;

import static minicp.cp.BranchingScheme.firstFail;
import static minicp.cp.Factory.*;

import java.util.Arrays;
import java.util.stream.IntStream;

### Formal Representation of the DARP Instance

Each DARP instance consists of a list of $n$ requests. A request is a triple $(p, d, t)$ where $p$ is the pick-up location, $d$ is the drop-off location and $t$ is the latest time at which the drop-off must be done. The problem instance also needs a transition time function $\text{transition}(a,b)$ which gives the travel time from position $a$ to position $b$.

For this CP model, we will consider that pick-up or drop-off can only happen in a limited number of locations. We store the coordinates of these locations in an array called `coords`. 

```java
// coordinates of the positions involved in this problem
int [][] coords = new int[][] {
        {10,20}, // 0
        {30,30}, // 1
        {50,40}, // 2
        {40,10}, // 3
        {50,20}, // 4
        {25,30}, // 5
        {10,5}   // 6
};
```

For simplicity, we let the transition times be the Euclidean distances of the coordinates $\text{transition}((x_1,y_1),(x_2,y_2)) = \sqrt{(x_1-x_2)^2 + (y_1-y_2)^2}$. We compute them for all possible transitions in this instance and store these in a 2D array.

```java
// compute the symmetrical transition times as Euclidian distances
int [][] transitions = new int[coords.length][coords.length];
for (int i = 0; i < coords.length; i++) {
    for (int j = 0; j < coords.length; j++) {
        int x1 = coords[i][0];
        int y1 = coords[i][1];
        int x2 = coords[j][0];
        int y2 = coords[j][1];
        transitions[i][j] = (int) Math.sqrt(Math.pow(x1-x2,2.0)+Math.pow(y1-y2,2.0));
    }
    System.out.println(Arrays.toString(transitions[i]));
}
```

The requests can then be represented by int arrays of three elements `{fromPos,toPos,deadline}`, with `fromPos` and `toPos` the indices of the locations in the `coords` array.

```java
// the requests defined as {fromPos,toPos,deadline}
int [][] requests = new int[][] {
        {3,5,120},
        {6,2,200},
        {5,1,100},
        {1,6,60},
        {5,2,150},
        {6,3,150},
};
```

Finally, the vehicle has a limited capacity. In our case, it can transport up to two passengers at the same time.

```java
// capacity of the vehicle
int vehicleCapacity = 2;
```

Copy and paste these statements into the code block below, and run them by clicking the arrow on the left.

In [ ]:
// DARP instance definition


### miniCP Model

We will now write the miniCP model to solve the DARP instance. Don't hesitate to consult the [miniCP API documentation](https://minicp.bitbucket.io/apidocs/) if something is not clear. First of all, we need to instanciate a miniCP solver. Then, we will define the different variables in our model, and the constraints between them.

```java
Solver cp = makeSolver();
```

#### Making Sure We Visit Everything

In a DARP, we have $n$ requests to fulfill, i.e. we have to leave the depot location, then visit (in some non-specified order) the $n$ pick-up locations and $n$ drop-off locations. Each of these $2n+1$ stops is called a visit, and we will denote the set of all visits by $V = \{0, \dots, 2n\}$. To model the route that our driver will take, we will list locations of each visit $i$ as `v[i]`.

```java
int n = requests.length;
int [] v = new int[2*n+1];
v[0] = 0; // departure from the depot (position index 0)
// [1..n+1] = pickups
// [n+1..2n+1] = delivery
for (int i = 0; i < n; i++) {
    v[1+i] = requests[i][0];
    v[n+1+i] = requests[i][1];
}
```

A solution to the DARP instance will consist of a total ordering of these visits — e.g. $1, 11, 2, \dots, 8$ — which respects all the constraints of the problem.

For each visit $i$, the integer variable `pred[i]` with domain $V = \{0, \dots, 2n\}$ models its direct predecessor in the route. By convention, the predecessor of the first visit of the vehicle (the depot) will be the last visit of the route.

For each visit $i$, `succ[i]` models its direct successor. The predecessors and successors are kept consistent via the constraint: `succ[pred[i]] == i`. To ensure that the order of the visits form a Hamiltonian circuit — i.e. that every visit is performed once and once only — we use the [`Circuit`](https://minicp.bitbucket.io/apidocs/minicp/engine/constraints/Circuit.html) constraint.

```java
IntVar [] succ = makeIntVarArray(cp, 2*n+1,2*n+1);
IntVar [] pred = makeIntVarArray(cp, 2*n+1,2*n+1);

// channeling between pred and succ vectors
for (int i = 0; i < succ.length; i++) {
    // succ[pred[i]] == i
    cp.post(equal(elementVar(succ,pred[i]),i));
}

cp.post(new Circuit(pred));
```

In that code block, the function `elementVar(IntVar[] array, IntVar y)` returns a fresh `IntVar` which represents the value of `array[y]`. It is a function we will use many times in the CP model, and it is defined as:

```java
public static IntVar elementVar(IntVar[] array, IntVar y) {
    Solver cp = y.getSolver();
    int min = IntStream.range(0, array.length).map(i -> array[i].min()).min().getAsInt();
    int max = IntStream.range(0, array.length).map(i -> array[i].max()).max().getAsInt();
    IntVar z = makeIntVar(cp, min,max);
    cp.post(new Element1DVar(array, y, z));
    return z;
}
```

Copy the code in the code block below and move on to the next section.

In [ ]:
// CP model: visits, prev and succ

#### Every Visit at the Right Time

We have modeled that all visits must be made, but we are still lacking the notions of _time_ and _capacity_. We will now model the time constraints: pick-ups have to be made before their drop-off, which have to be made before their deadline.

For each visit $i$, the integer variable `time[i]` will model the time at which that visit is done. By convention, the departure time from the depot (the start of the route) is 0, `time[0] = 0`.

```java
IntVar [] time = makeIntVarArray(cp, 2*n+1,500);
// departure time from the depot =  0
cp.post(equal(time[0],0));
```

The time at which we make visit $i$ is the time at which the preceding visit `pred[i]` was made, plus the transition time between `v[pred[i]]` and `v[i]`, i.e. `time[i] = time[pred[i]] + transition[ v[pred[i]] ][ v[i] ]`.

```java
// visit time update
for (int i = 1; i < 2*n+1; i++) {
    // time[i] = time[pred[i]] + transition[pred[i]][i]
    IntVar tPredi = elementVar(time, pred[i]);
    IntVar posPredi = element(v, pred[i]);
    IntVar tt = element(transitions[v[i]],posPredi);
    cp.post(equal(time[i],sum(tPredi,tt)));
}
```

We also must make sure that the pick-up of a request is done before its drop-off `time[pick-up] ≤ time[drop-off]`, and that the drop-off is done before the deadline `time[drop-off] ≤ deadline`.

```java
// precedence between pickup and delivery + deadlines
for (int i = 0; i < n; i++) {
    // pickup before delivery)
    cp.post(lessOrEqual(time[i+1],time[n+i+1]));
    // delivery before the deadline
    cp.post(lessOrEqual(time[n+i+1],requests[i][2]));
}
```

In [ ]:
// CP model: time constraints


#### Vehicle Capacity

The final constraint of our model ensures that we do not pick-up more passengers than the capacity of our vehicle. The integer variable `load[i]` will model the load of vehicle (number of passengers on board) after having done visit $i$.

```java
IntVar [] load = makeIntVarArray(cp, 2*n+1,vehicleCapacity+1);
```

After picking up a passenger, this load increases by one `load[i] = load[pred[i]] + 1`. After dropping off a passenger, the load decreases by one `load[i] = load[pred[i]] - 1`.

```java
// load update after a pickup (+1)
for (int i = 1; i <= n; i++) {
    // load[i] = load[[pred[i]] + 1
    IntVar loadPred = elementVar(load, pred[i]);
    cp.post(equal(load[i], plus(loadPred, 1)));
}

// load update after a delivery (-1)
for (int i = n+1; i <= 2*n; i++) {
    // load[i] = load[[pred[i]] - 1
    IntVar loadPred = elementVar(load, pred[i]);
    cp.post(equal(load[i], plus(loadPred, -1)));
}
```


In [ ]:
// CP model: load constraints


### Searching for a solution

Every necessary constraint has been modelled. We can now use the miniCP solver to find all the solutions to the DARP instance. We will use a simple [Depth-First Search](https://minicp.bitbucket.io/apidocs/minicp/search/DFSearch.html), but you are free to implement your own custom search with appropriate heuristics.

Run the code block below to make sure that your CP model is correct.

In [4]:
DFSearch dfs = makeDfs(cp, firstFail(pred));

dfs.onSolution(() -> {
    System.out.println("solution");
    System.out.println("succ:"+Arrays.toString(succ));
    System.out.println("pred:"+Arrays.toString(pred));
    System.out.println("time:"+Arrays.toString(time));
    System.out.println("load:"+Arrays.toString(load));

    int curr = 0;
    for (int i = 0; i < succ.length; i++) {
        System.out.println("visiting position:"+positionIdx[curr]+" at time:"+time[curr]+" load:"+load[curr]);
        curr = succ[curr].min();
    }

});

// search for the first feasible solution
SearchStatistics stats = dfs.solve(statistics -> {
    return statistics.numberOfSolutions() > 0;
});

System.out.println(stats);

solution
succ:[{3}, {7}, {10}, {9}, {2}, {8}, {12}, {5}, {11}, {4}, {6}, {0}, {1}]
pred:[{11}, {12}, {4}, {0}, {9}, {7}, {10}, {1}, {5}, {3}, {2}, {8}, {6}]
time:[{0}, {85}, {55}, {18}, {23}, {110}, {55}, {110}, {136}, {23}, {55}, {136}, {85}]
load:[{0}, {2}, {2}, {1}, {1}, {2}, {2}, {1}, {1}, {0}, {1}, {0}, {1}]
visiting position:0 at time:{0} load:{0}
visiting position:5 at time:{18} load:{1}
visiting position:1 at time:{23} load:{0}
visiting position:1 at time:{23} load:{1}
visiting position:6 at time:{55} load:{2}
visiting position:6 at time:{55} load:{1}
visiting position:6 at time:{55} load:{2}
visiting position:3 at time:{85} load:{1}
visiting position:3 at time:{85} load:{2}
visiting position:5 at time:{110} load:{1}
visiting position:5 at time:{110} load:{2}
visiting position:2 at time:{136} load:{1}
visiting position:2 at time:{136} load:{0}

	#choice: 31854
	#fail: 15923
	#sols : 1
	completed : false

